In [1]:
import pandas as pd
import numpy as np

### Data sources
1. sample_id_to_barcode - from make_mapping_file Notebook - from original spreadsheet that contains grid of sequence ids and barcodes
2. nextclade_report- from Nextclade run in Galaxy history 'batch1 consensus genomes 2' (consensus genomes with non-N at depth >= 20 and AF >= 0.8)
3. sequence_mapping_report - exported from MultiQC table of Galaxy 'batch 1' analysis
4. spreadsheet_with_metadata - Excel spreadsheet with patient metadata and sample IDs

In [64]:
barcode_sampleid_mapping_df = pd.read_csv('data/sample_id_to_barcode.csv', dtype={'sample_id': 'str'})
nextclade_df = pd.read_csv('data/nextclade_report.tsv', delimiter='\t')
nextclade_qc_columns = [ col for col in nextclade_df.columns if col.startswith('qc.') or col.startswith('total') ]
nextclade_qc_df = pd.read_csv('data/nextclade_report.tsv', delimiter='\t', usecols=['seqName', 'clade'] + nextclade_qc_columns) 
sequence_mapping_df = pd.read_csv('data/sequence_mapping_report.tsv', delimiter='\t', 
                                  usecols=['Sample Name', '≥ 30X', 'Median cov', 'Mean cov', 'M Reads Mapped']).rename({'Sample Name': 'seqName'}, axis=1)

In [68]:
nextclade_qc_df.set_index('seqName')

,clade,qc.overallScore,qc.overallStatus,totalSubstitutions,totalDeletions,totalInsertions,totalFrameShifts,totalAminoacidSubstitutions,totalAminoacidDeletions,totalAminoacidInsertions,...,qc.frameShifts.frameShifts,qc.frameShifts.totalFrameShifts,qc.frameShifts.frameShiftsIgnored,qc.frameShifts.totalFrameShiftsIgnored,qc.frameShifts.score,qc.frameShifts.status,qc.stopCodons.stopCodons,qc.stopCodons.totalStopCodons,qc.stopCodons.score,qc.stopCodons.status
seqName,,,,,,,,,,,,,,,,,,,,,
barcode50,21K (Omicron),1959.456187,bad,60,10,34,4,32,3,3,...,"ORF1a:698-1377,ORF1a:1672-2333,ORF1a:4366-4401...",4,NaN,0,300.0,bad,NaN,0,0.0,good
barcode72,21L (Omicron),97.170374,mediocre,64,18,2,1,47,6,0,...,ORF1b:2445-2696,1,NaN,0,75.0,mediocre,NaN,0,0.0,good
barcode80,21L (Omicron),93.009520,mediocre,60,18,1,1,43,6,0,...,N:143-420,1,NaN,0,75.0,mediocre,NaN,0,0.0,good
barcode71,21L (Omicron),126.902689,bad,68,18,2,1,47,6,0,...,ORF1b:2487-2696,1,NaN,0,75.0,mediocre,NaN,0,0.0,good
barcode64,21L (Omicron),150.609290,bad,57,18,1,1,46,6,0,...,ORF1b:2577-2696,1,NaN,0,75.0,mediocre,N:135,1,75.0,mediocre
barcode76,21K (Omicron),1981.835971,bad,45,14,8,4,11,2,0,...,"ORF1a:2337-2389,ORF1a:3366-4401,ORF1b:750-2696...",4,NaN,0,300.0,bad,NaN,0,0.0,good
barcode89,21K (Omicron),5065.413172,bad,41,2,15,4,6,0,1,...,"ORF1a:456-1310,ORF1a:3422-4401,ORF1b:823-2696,...",4,NaN,0,300.0,bad,NaN,0,0.0,good
barcode78,21L (Omicron),406.504993,bad,61,18,9,1,35,3,0,...,ORF1a:27-4401,1,NaN,0,75.0,mediocre,NaN,0,0.0,good
barcode73,21K (Omicron),0.000000,good,53,39,9,0,46,16,1,...,NaN,0,NaN,0,0.0,good,NaN,0,0.0,good


In [44]:
# NB
# convert the metadata from CIFs to metadata relevant to GISAID

def convert_gender(g):
    if g == 'm':
        return 'Male'
    elif g == 'f':
        return 'Female'
    else:
        return 'Unknown'
    
def convert_district(d):
    d = d.replace("'", "")
    if ' ' in d:
        fields = d.split()
        d = ' '.join([field.capitalize() for field in fields])
    else:
        d = d.capitalize()
    if d == 'Butha Bothe':
        d = 'Butha Buthe'
    return d

patient_metadata = pd.read_excel('data/spreadsheet_with_metadata.xlsx', sheet_name='Sheet3', 
                                 converters = {'Sample ID': lambda s: str(s),
                                               'Gender':  convert_gender,
                                               'District': convert_district}).rename({'Sample ID': 'sample_id'}, axis=1)
patient_metadata.columns
patient_metadata_subset = patient_metadata[['sample_id', 'specimen collection', 'Specimen Type', 'District',
                                            'qPCR Result Date', 'CT Value N GENE', 'CT Value ORF GENE']]

# this is a subset oriented around submission to GISAID - less relevant for assessing sample quality
# patient_metadata_subset = patient_metadata[['sample_id', 'specimen collection', 'Specimen Type', 'Purpose of Sampling', 'Age', 'Gender', 'District',
#                                             'qPCR Result Date', 'CT Value N GENE', 'CT Value ORF GENE']]
# now = pd.Timestamp('now')
# patient_metadata_subset.insert(len(patient_metadata_subset.columns), 'age', (now - patient_metadata_subset['Age']).astype('<m8[Y]').replace(-1, np.nan))
# patient_metadata_subset = patient_metadata_subset.drop('Age', axis=1)
patient_metadata_subset.insert(len(patient_metadata_subset.columns), 'days to PCR result', patient_metadata_subset['qPCR Result Date'] - patient_metadata_subset['specimen collection'])

In [71]:
patient_info_with_barcodes = patient_metadata_subset.set_index('sample_id').join(barcode_sampleid_mapping_df.set_index('sample_id')).reset_index()
patient_info_with_barcodes_and_mapping_info = patient_info_with_barcodes.set_index('seqName').join(sequence_mapping_df.set_index('seqName'))
combined_df = patient_info_with_barcodes_and_mapping_info.join(nextclade_qc_df.set_index('seqName')).reset_index()
combined_df.to_excel('data/combined_qc_info.xlsx')